In [10]:
import sys
sys.path.append("../")

from itops.db.sqlite.sqlitehelper import SQLiteDatabaseHelper
from config.configs import CONFIGS
import uuid

In [11]:
db_helper = SQLiteDatabaseHelper("itops.db")

In [12]:
db_helper.connect()

db_helper.execute_query("""
                        DROP TABLE cluster_data """)
        
# Create a table
db_helper.execute_query("""
CREATE TABLE IF NOT EXISTS cluster_data (
                    RUN_ID VARCHAR(500),
                    RUN_NAME VARCHAR(500), 
                    CATEGORY VARCHAR(500), 
                    INPUT_FILE_NAME VARCHAR(500), 
                    INSIGHTS_FILE_NAME VARCHAR(500),
                    CLUSTER_ID VARCHAR(500), 
                    CLUSTER_NAME VARCHAR(500), 
                    PARENT_CLUSTER_ID VARCHAR(500), 
                    PARENT_CLUSTER_NAME VARCHAR(500)
                    )
""")

Connected to database: itops.db
Query executed successfully.
Query executed successfully.


In [13]:
db_helper.close_connection()

Connection closed.


In [14]:
db_helper.connect()
records = db_helper.fetch_all("SELECT * FROM cluster_data")
db_helper.close_connection()

Connected to database: itops.db
Fetched 0 rows.
Connection closed.


In [15]:
for record in records:
    print(record)

In [16]:
records

[]

In [17]:
def query_helper(query,db_type = "DUCKDB"):

        if db_type == "DUCKDB":
            query = query.replace('%s', '?')
        
        return(query)

In [20]:
def insert_cluster_data(run_name, category, input_file_name, insights_file_name, cluster_name, parent_cluster_name):
        # Establish a connection to the database
        db_helper.connect()
        print("CONNECTED to the database")

        # Define the insert query
        insert_query = """
        INSERT INTO cluster_data (RUN_ID,RUN_NAME, 
                                CATEGORY, 
                                INPUT_FILE_NAME, 
                                INSIGHTS_FILE_NAME, 
                                CLUSTER_ID,
                                CLUSTER_NAME, 
                                PARENT_CLUSTER_ID,
                                PARENT_CLUSTER_NAME)
        VALUES (%s, %s, %s, %s, %s, %s,%s, %s, %s)
        """

        insert_query = query_helper(insert_query)
        print(insert_query)

        RUN_ID =  str(uuid.uuid4())
        CLUSTER_ID =  str(uuid.uuid4())
        PARENT_CLUSTER_ID =  str(uuid.uuid4())
        
        # Prepare the data tuple
        data = (RUN_ID,run_name, 
                category, input_file_name, 
                insights_file_name, 
                CLUSTER_ID,cluster_name, 
                PARENT_CLUSTER_ID,parent_cluster_name)

        try:
            # Execute the insert query with the provided data
            db_helper.execute_query(insert_query, data)
            print("Data inserted successfully.")
        except Exception as e:
            print(f"An error occurred: {e}")
        finally:
            # Ensure the database connection is closed
            db_helper.close_connection()

In [21]:
insert_cluster_data(run_name ="RUN001", 
                    category = "CATEGORY001", 
                    input_file_name = "FILE001.TXT", 
                    insights_file_name = "INSIGHTS.TXT", 
                    cluster_name = "CLUSTER001", 
                    parent_cluster_name = "PARENTCLUSTER001")

Connected to database: itops.db
CONNECTED to the database

        INSERT INTO cluster_data (RUN_ID,RUN_NAME, 
                                CATEGORY, 
                                INPUT_FILE_NAME, 
                                INSIGHTS_FILE_NAME, 
                                CLUSTER_ID,
                                CLUSTER_NAME, 
                                PARENT_CLUSTER_ID,
                                PARENT_CLUSTER_NAME)
        VALUES (?, ?, ?, ?, ?, ?,?, ?, ?)
        
Query executed successfully.
Data inserted successfully.
Connection closed.


In [22]:
db_helper.connect()
records = db_helper.fetch_all("SELECT * FROM cluster_data")
db_helper.close_connection()

Connected to database: itops.db
Fetched 1 rows.
Connection closed.


In [23]:
records

[('dee527b7-402e-4af9-bc38-9c1be62f9f64',
  'RUN001',
  'CATEGORY001',
  'FILE001.TXT',
  'INSIGHTS.TXT',
  '6c6ad4ca-ec91-42c9-957d-45280e1edf63',
  'CLUSTER001',
  'ef7c4d5c-accb-4274-a421-879349d1c995',
  'PARENTCLUSTER001')]